In [2]:

import pandas as pd
import numpy as np

import json
import os

import re
import pymongo

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

In [10]:
boston_drinks = pd.read_csv('cocktails_cleaned.csv')
boston_drinks.head()


,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,special ingredients,instructions,glass,glass-size (oz)
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Cherry,Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.,Old-Fashioned Glass,6.0
1,Fort Lauderdale,Cocktail Classics,1 1/2 oz,Light Rum,1/2 oz,Sweet Vermouth,1/4 oz,Juice of Orange,1/4 oz,Lime Juice,NaN,NaN,NaN,NaN,Orange Slice,Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.,Old-Fashioned Glass,6.0
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Apple Slice,Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.,Old-Fashioned Glass,6.0
3,Cuban Cocktail No. 1,Cocktail Classics,2 oz,Light Rum,1/2 oz,Powdered Sugar,1/2 oz,Juice of a Lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6.0
4,Cool Carlos,Cocktail Classics,1 1/2 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Pineapple Slice, Orange Slice, Cherry","Mix all ingredients except curacao with ice, shake well. Pour into collins glass and top with curacao. Garnish with a pineapple slice, an orange slice, and a cherry.",Collins Glass,14.0


In [11]:
boston_drinks = boston_drinks.fillna('')

In [12]:
cols = list(boston_drinks)
for i, col in enumerate(cols):
    print(i, col)

0 name
1 category
2 measurement-1
3 ingredient-1
4 measurement-2
5 ingredient-2
6 measurement-3
7 ingredient-3
8 measurement-4
9 ingredient-4
10 measurement-5
11 ingredient-5
12 measurement-6
13 ingredient-6
14 special ingredients
15 instructions
16 glass
17 glass-size (oz)


In [15]:
recipes = []
for index, row in boston_drinks.iterrows():
    name = boston_drinks.loc[index, 'name']
    this_recipe = {}
    this_recipe['name'] = name
    this_recipe['category'] = boston_drinks.loc[index, cols[1]]
    this_recipe['ingredients'] = []
    this_recipe['special_ingredients'] = boston_drinks.loc[index, cols[14]]
    this_recipe['instructions'] = boston_drinks.loc[index, cols[15]]
    if (boston_drinks.loc[index, cols[2]] != '') and (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[2]]) + ' ' + str(boston_drinks.loc[index, cols[3]]))
    elif (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[4]] != '') and (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[4]]) + ' ' + str(boston_drinks.loc[index, cols[5]]))
    elif (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[6]] != '') and (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[6]]) + ' ' + str(boston_drinks.loc[index, cols[7]]))
    elif (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[7]]))
    if (boston_drinks.loc[index, cols[8]] != '') and (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[8]]) + ' ' + str(boston_drinks.loc[index, cols[9]]))
    elif (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[9]]))
    if (boston_drinks.loc[index, cols[10]] != '') and (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[10]]) + ' ' + str(boston_drinks.loc[index, cols[11]]))
    elif (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[11]]))
    if (boston_drinks.loc[index, cols[12]] != '') and (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[12]]) + ' ' + str(boston_drinks.loc[index, cols[13]]))
    elif (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[13]]))
    this_recipe['glass_type'] = boston_drinks.loc[index, cols[16]]
    this_recipe['glass_size'] = str(boston_drinks.loc[index, cols[17]])
    recipes.append(this_recipe)

In [16]:
recipes

[{'name': 'Gauguin',
  'category': 'Cocktail Classics',
  'ingredients': ['2 oz  Light Rum',
   '1 oz  Passion Fruit Syrup',
   '1 oz  Lemon Juice',
   '1 oz  Lime Juice'],
  'special_ingredients': 'Cherry',
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0'},
 {'name': 'Fort Lauderdale',
  'category': 'Cocktail Classics',
  'ingredients': ['1 1/2 oz  Light Rum',
   '1/2 oz  Sweet Vermouth',
   '1/4 oz  Juice of Orange',
   '1/4 oz  Lime Juice'],
  'special_ingredients': 'Orange Slice',
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0'},
 {'name': 'Apple Pie',
  'category': 'Cordials and Liqueurs',
  'ingredients': ['3 oz  Apple schnapps', '1 oz  Cinnamon schnapps'],
  'special_ingredients': 'Apple

In [17]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.cocktail_db

collection = mongoDb.recipe_dump

collection.insert_many(recipes)

In [29]:
cocktail_db_response = collection.find({}, {'_id': False})


In [30]:
recipes = []
for recipe in cocktail_db_response:
    recipes.append(recipe)


In [31]:
recipes

[{'name': 'Gauguin',
  'ingredients': ['2 oz  Light Rum',
   '1 oz  Passion Fruit Syrup',
   '1 oz  Lemon Juice',
   '1 oz  Lime Juice'],
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Fort Lauderdale',
  'ingredients': ['1 1/2 oz  Light Rum',
   '1/2 oz  Sweet Vermouth',
   '1/4 oz  Juice of Orange',
   '1/4 oz  Juice of a Lime'],
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Apple Pie',
  'ingredients': ['3 oz  Apple schnapps',
   '1 oz  Cinnamon schnapps',
   ' Apple slice'],
  'instructions': 'Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Cuban C